In [ ]:
LOG_FILE="/Users/law/repos/ma/benchmark-runs/micro_measure_2/micro_measure.log"

In [ ]:
import re
from collections import defaultdict

RECV_LINE = re.compile(r"Avg receiving  time: (\d+) ns.")
PROC_TIME = re.compile(r"Avg processing time: (\d+) ns.")
WM_PROC_LINE = re.compile(r"Watermark processing took (\d+) ns.")
WM_SEND_LINE = re.compile(r"Watermark sending took (\d+) ns.")
SESSION_LINE = re.compile(r"Session start took (\d+) ns.")
MAX_LINE = "TUMBLING,1000 MAX"
AVG_LINE = "TUMBLING,1000 M_AVG"
MEDIAN_LINE = "TUMBLING,1000 M_MEDIAN"

FUNCTIONS = ["median", "max", "avg"]

MATCHERS = [
    (RECV_LINE, "recv"), (PROC_TIME, "proc"), 
    (WM_PROC_LINE, "wm-proc"), (WM_SEND_LINE, "wm-send"),
    (SESSION_LINE, "session"),
]

def get_processing_times(log_file):
    with open(log_file) as f:
        times = {}
        for fn in FUNCTIONS:
            times[fn] = {}
            for _, key in MATCHERS:
                times[fn][key] = []

        current_function = None
        for line in f:
            if MAX_LINE in line:
                current_function = "max"
            if AVG_LINE in line:
                current_function = "avg"
            if MEDIAN_LINE in line:
                current_function = "median"

            for matcher, key in MATCHERS:
                match = matcher.match(line)
                if match is not None:
                    duration = int(match.group(1))
                    times[current_function][key].append(duration)
    return times


ALL_TIMES = get_processing_times(LOG_FILE)
print(ALL_TIMES)

In [ ]:
CLEAN_TIMES = {}
for fn, proc_times in ALL_TIMES.items():
    CLEAN_TIMES[fn] = {}
    for key, times in proc_times.items():
        num_times = len(times)
        cut_off = max(1, int(num_times * 0.05))
        cut_times = times[cut_off:-cut_off] 
        avg = sum(cut_times) / len(cut_times)
        print(f"{fn} - {key}: {avg:.2f} ns ({num_times} values)")
        CLEAN_TIMES[fn][key] = cut_times
        CLEAN_TIMES[fn][f"avg-{key}"] = avg

print("\n======\n")
        
for fn, proc_times in CLEAN_TIMES.items():
    recv = proc_times['avg-recv']
    proc = proc_times['avg-proc']
    avg_both = recv + proc
    one_sec_ns = 1_000_000_000
    max_per_sec = one_sec_ns / avg_both
    # We measure 33ns duration for each call to System.nanoTime(), so we add it to second
    syscall_ns = 33 * 4
    clean_sec_ns = one_sec_ns + (max_per_sec * syscall_ns)
    max_per_sec_clean = clean_sec_ns / avg_both
    print(f"{fn}: {max_per_sec:.0f} events/s.")
    print(f"{fn}: {max_per_sec_clean:.0f} clean events/s.")
        

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True, 'pgf.rcfonts' : False})

In [ ]:
def plot_proc_times(fn, proc_times):
    entries = sorted(proc_times.items())
    keys, times = zip(*entries)
    N = len(keys)
    plt.boxplot(times, showfliers=False)
    plt.xticks(range(1, len(keys) + 1), keys)
    plt.yscale('log')
    plt.title(fn)
    plt.show()

for fn in FUNCTIONS:
    plot_proc_times(fn, ALL_TIMES[fn])